# Earth Engine Communicator 
API DOC: https://developers.google.com/earth-engine/api_docs#eefeatureget

## Import Packages
pip install earthengine-python-api??
conda install ipykernel
python -m ipykernel install --user
conda install -c conda-forge geopandas
conda install -c conda-forge ipyleaflet 
conda install -c conda-forge shapely 
conda install -c conda-forge scikit-image

In [1]:
import ee
import datetime
import IPython.display
from IPython.display import Image
import bqplot
import ipywidgets
import pprint
pp = pprint.PrettyPrinter(indent=2) # for printing pretty idk what it is... print with pp.pprint(print stuff)
import pandas as pd
import geopandas as gpd
import matplotlib.dates as mdates
from matplotlib import dates
%matplotlib inline
from shapely.geometry import shape
import skimage
import traitlets
import dateutil.parser
import numpy as np
import ipyleaflet
from ipyleaflet import (LayersControl, basemaps, basemap_to_tiles, LayerGroup, Map, Polygon, GeoJSON)
import json
import os
import requests

# Connect to Earth Engine API
ee.Initialize()

## Data Viewer

In [2]:
#Get TileLayerurl from ee to plot on ipyleaflet
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

In [12]:
#Visualization Parameters
dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
germany_viz = GetTileLayerUrl(germany_outline.visualize())
#radolan_viz = GetTileLayerUrl(radolan.visualize(min=0,max=40,bands=['b1']))
clc_viz = GetTileLayerUrl(clc.visualize(min=0,max=7,palette=['FFE6FF','FFFFA8','FFFF00','E6E600','E68000','F2A64D','E6A600']))
clc_vector_viz = GetTileLayerUrl(clc_vector.draw(color='red').visualize())
#surface_moisture_viz = GetTileLayerUrl(surface_moisture.visualize(min=0,max=28,palette=['0300ff', '418504', 'efff07', 'efff07', 'ff0303']))
random_points_viz = GetTileLayerUrl(random_points.draw(color='blue').visualize())

# Create layer group
layer_group = LayerGroup(layers=(ipyleaflet.TileLayer(url=clc_viz,name='Corine Land Cover 2012'), 
                                 ipyleaflet.TileLayer(url=germany_viz, name='german boundary'), 
                                 ipyleaflet.TileLayer(url=clc_vector_viz, name='clc_vector'),
                                 ipyleaflet.TileLayer(url=random_points_viz, name='rand point')
                                 #ipyleaflet.TileLayer(url=surface_moisture_viz, name='GSMD surface moisture')
                                ))
#Map options
center,zoom = [12.727661, 51.229822],4

#Interactive Visualizations
map1 = ipyleaflet.Map(layer =dark_matter_layer, center=center, zoom=zoom, layout={'height' : '400px'})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1.add_layer(layer_group)
map1.add_control(LayersControl())
map1


NameError: name 'clc_vector' is not defined

In [ ]:
#Get Coordinates from Drawn Geometry
dc.last_draw

## AOI, Boundarys, Geometrys, Features, Points

In [11]:
# AOI
ried = ee.FeatureCollection('users/tillmueller1990/ried_roi')
ried_225_222 = ee.FeatureCollection('users/tillmueller1990/ried_225_222')
germany = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na','Germany'))

# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte();
# Paint all the polygon edges with the same number and width, display.
germany_outline = empty.paint(germany, 1, 3)

#Sample Points Germany 500
random_points = ee.FeatureCollection.randomPoints(clc.geometry(), 5000)

# Datasets 

In [4]:
#Filterparameters 
# Dates
startTime = datetime.datetime(2009,1,1)
endTime = datetime.datetime(2019,1,1)

## Precipitaion Data 

In [5]:
#Load Radolan from Assest 
radolan = ee.ImageCollection('users/tillmueller1990/radolan/r_2009-2019')

#Load GSMAP Hourly Precipitation
gsmap = ee.ImageCollection('JAXA/GPM_L3/GSMaP/v6/operational')

#CHIRPS 5km daily precipitation 
chirps  = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')

## Soil Moisture

Products provided by NASA, ESA, GEE ....

In [6]:
#NASA-USDA Global Soil Moisture Data / 3 days / 2010- 2019 / 0.25 arc degrees / SMOS lvl2 integrated into two-layer palmer model
SMOS = ee.ImageCollection('NASA_USDA/HSL/soil_moisture')
surface_moisture = SMOS.select('ssm') #units mm, min=0 max=25
subsurface_moisture = SMOS.select('susm') #units: mm min=0, max=275
moisture_profile = SMOS.select('smp') #units: fraction, min0, max 1

#NASA-USDA SMAP Global Soil Moisture Data / 3 days / 2015 - 2019 / 0.25 arc degrees / SMAP level 3 + two-layer Palmer
SMAP = ee.ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture')

#GLDAS-2.1: Global Land Data Assimilation System / 3 hours / 0.25 arc degrees / 2000 - 2019 / 

Sentinel 1 backscatter raw data convertet to reflectivity range between 0 and 1 indicator for high reflectivity = high water volume content and vis verse 

In [71]:
#Load Sentinel 1 and filter data
def load_dataset(ImageCollection_ID,begin,end,aoi):
    ic = ee.ImageCollection(ImageCollection_ID).filterDate(begin,end).filterBounds(aoi)
    return ic

def filter_sentinel1(ImageCollection,polarisation,instrumentMode,resolution):
    ic = ImageCollection.filter(ee.Filter.listContains('transmitterReceiverPolarisation',polarisation)).filter(ee.Filter.eq('instrumentMode',instrumentMode)).filterMetadata('resolution_meters','equals', resolution)
    return ic

def seperate_look_angels(ImageCollection,polarisation):
    Ascending = ImageCollection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).select(polarisation)
    Descending = ImageCollection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).select(polarisation)
    return Ascending,Descending

def seperate_tiles(ImageCollection,tiles):
    tile_list = [x]

def get_geometry(ImageCollection):
    geometry = ImageCollection.geometry().getInfo()
    #geometry = [x for x in geometry['coordinates']]
    return geometry

def show_tiles(ImageCollection):
    geometry = get_geometry(ImageCollection)
    geometry_list = list(geometry['coordinates'])
    flattened_list = [y for x in geometry_list for y in x] #De flatter list 
    unique_list = []
    unique_list = [x for x in flattened_list if x not in unique_list]
    print("different tiles: ",len(unique_list))
    m = Map(center=(49.6252978589571, 8.34580993652344), zoom=7)
    geo_json = GeoJSON(data=geometry, style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
    m.add_layer(geo_json)
    dc = ipyleaflet.DrawControl()
    m.add_control(dc)
    m.add_control(LayersControl())
    return m
    
def get_properties(ImageCollection):
    features = ImageCollection.getInfo()['features']
    dict_list = []
    for f in features:
        prop = f['properties']
        dict_list.append(prop)
    df = pd.DataFrame.from_records(dict_list).drop(['system:footprint','transmitterReceiverPolarisation'],axis=1)
    #Pandas Series of unique distinc values in df
    unique = df.nunique()
    im_id_list = [item.get('id') for item in ImageCollection.getInfo().get('features')]
    date_list = [datetime.datetime.strptime(x[35:43],'%Y%m%d') for x in im_id_list]
    #property_names = list(df.columns.values) 
    return unique, im_id_list, date_list

def make_mosaic(date):
    #Cast objects
    #date = ee.Number(date)
    date = ee.Date(date['value'])
    #print(date)
    #newList = ee.List([])
    #print(date)
    filterCollection = VV_Ascending.filterDate(date, date.advance(1,'day'))
    #print(filterCollection.size().getInfo())
    #Make the mosaic
    image = ee.Image(filterCollection.mosaic()).copyProperties(filterCollection.first(),["system:time_start"])
    #newList.add(image)
    #Add the mosaic to a list only if the collection has images
    #print(image.getInfo())
    #return ee.List(ee.Algorithms.If(filterCollection.size(), newList.add(image), newList))
    #print(image.getInfo())
    return image

def clip_aoi(ImageCollection):
    im = ImageCollection.clip(ried_225_222)
    return im

def plot_image(ImageCollection):
    m = Map(center=(49.6252978589571, 8.34580993652344), zoom=7)
    ic = GetTileLayerUrl(ImageCollection.first().visualize())
    m.add_layer(ic)
    dc = ipyleaflet.DrawControl()
    m.add_control(dc)
    m.add_control(LayersControl())
    return m

#Time of interest
begin = ee.Date.fromYMD(2015,1,1)
end = ee.Date.fromYMD(2016,1,1)
date_range = end.difference(begin, 'day')
#Source dataset
sentinel1 = load_dataset('COPERNICUS/S1_GRD',begin,end,ried_225_222)
#Filter dataset for High resolution and Vertical transmitt vertical receive
sentinel1_VV = filter_sentinel1(sentinel1,'VV','IW',10)
#Filter for different look angles
VV_Ascending,VV_Descending = seperate_look_angels(sentinel1_VV,'VV')

#Get geometry for plotting different tiles
#geometry_vv = get_geometry(VV_Ascending)
 #Plot Tiles

#Get list of ids,dates and unique count of prop
unique, im_id_list, date_list = get_properties(VV_Ascending)
date_list = ee.List([ee.Date(x) for x in date_list])
#pp.pprint(unique)
VV_aoi = VV_Ascending.map(clip_aoi)
#pp.pprint(VV_aoi.first().area().divide(1000 * 1000))
area = VV_Descending.first().multiply(ee.Image.pixelArea()).divide(1000*1000)
#Reducing the statistics for your study area
stat = area.reduceRegion(ee.Reducer.sum(),ried_225_222,100)
print('Vegetation Area (in sq.km)', stat.getInfo());
-6000 = normal 
80 sehr wenig 
pp.pprint(area.getInfo())
#show_tiles(VV_Ascending)
#plot_image(VV_aoi)


Vegetation Area (in sq.km) {'VV': -6374.363046381469}
{ 'bands': [ { 'crs': 'EPSG:32632',
               'crs_transform': [ 10.0,
                                  0.0,
                                  383818.24257877335,
                                  0.0,
                                  -10.0,
                                  5605230.681458753],
               'data_type': {'precision': 'double', 'type': 'PixelType'},
               'dimensions': [28797, 21300],
               'id': 'VV'}],
  'type': 'Image'}


In [ ]:
"""
newList = ee.List([])

for date in date_list.getInfo():
    mosaic = ee.ImageCollection(ee.List(make_mosaic(date)))
    #print(mosaic.getInfo())
    #print(mosaic)
    #print(newList)
newcol = ee.ImageCollection(newList)
#newcol = ee.ImageCollection(ee.List(date_list.iterate(make_mosaic(date), ee.List([]))))
#pp.pprint(newcol.getInfo())

#pp.pprint(newcol.getInfo()['features'])

#VV_Ascending_mosaic = make_mosaic(date_list)
#Iterate over the range to make a new list, and then cast the list to an imagecollection
#newcol = ee.ImageCollection([x]

#print(unique, im_id_list, date_list)
#pp.pprint(sentinel1_properties)
"""

In [232]:

print(newcol.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': []}


In [48]:


#Visualization Parameters
dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
germany_viz = GetTileLayerUrl(germany_outline.visualize())
#radolan_viz = GetTileLayerUrl(radolan.visualize(min=0,max=40,bands=['b1']))
#clc_viz = GetTileLayerUrl(clc.visualize(min=0,max=7,palette=['FFE6FF','FFFFA8','FFFF00','E6E600','E68000','F2A64D','E6A600']))
#clc_vector_viz = GetTileLayerUrl(clc_vector.draw(color='red').visualize())
#surface_moisture_viz = GetTileLayerUrl(surface_moisture.visualize(min=0,max=28,palette=['0300ff', '418504', 'efff07', 'efff07', 'ff0303']))
#random_points_viz = GetTileLayerUrl(random_points.draw(color='blue').visualize())
sentinel1_viz = GetTileLayerUrl(VV_aoi.first().visualize())

# Create layer group
layer_group = LayerGroup(layers=(#ipyleaflet.TileLayer(url=clc_viz,name='Corine Land Cover 2012'), 
                                 ipyleaflet.TileLayer(url=germany_viz, name='german boundary'),
                                 #ipyleaflet.TileLayer(url=clc_vector_viz, name='clc_vector'),
                                 #ipyleaflet.TileLayer(url=random_points_viz, name='rand point')
                                 #ipyleaflet.TileLayer(url=surface_moisture_viz, name='GSMD surface moisture')
                                 ipyleaflet.TileLayer(url=sentinel1_viz, name='Sentinel 1 VV ')
                                ))
#Map options
center,zoom = (49.6252978589571, 8.34580993652344),6

#Interactive Visualizations
map1 = ipyleaflet.Map(layer= basemap_to_tiles(basemaps.CartoDB.DarkMatter), center=center, zoom=zoom, layout={'height' : '400px'})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1.add_layer(layer_group)
map1.add_control(LayersControl())
map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## Corine Land Cover 

In [13]:
# Load last corine land cover image 
clc = ee.Image.load('COPERNICUS/CORINE/V18_5_1/100m/2012', -1)

# Mask areas where soil moisture measurements valid (farmland cat.:11-17)
clc_mask = clc.gte(11)and(clc.lte(17)) #binary map for updateMask
clc = clc.updateMask(clc_mask) #set mask for not Farmland

# Clip to extend of germany
clc = clc.clip(germany) 

# Feature Collection of clc farmland cat.: 11-17 with 350m resolution
clc_vector = clc.reduceToVectors(geometry=germany, crs=clc.projection(), scale=350, geometryType='polygon', eightConnected = False)
#Limit to first 5000 Features 
clc_vector = clc_vector.limit(5000)

## GLDAS-2.1: Global Land Data Assimilation System 
Global Land Data Assimilation System (GLDAS) ingests satellite and ground-based observational data products. Using advanced land surface modeling and data assimilation techniques, it generates optimal fields of land surface states and fluxes.

In [ ]:
GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')


## Analyse Hessisches Ried 
Only year by year otherwise memory exceeded
### SMOS

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2019,1,1),datetime.datetime(2020,1,1)
aoi = ried
sm = SMOS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi, scale=1000)
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    data_aoi['geometry'] = aoi.getInfo()['features']
    data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['properties']['ID'][23:29],'%y%m%d')
    df = pd.DataFrame(data_aoi)
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])
#### Iterate over all impages
#df_all = pd.DataFrame({})
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))
df_all.to_csv("SMOS_ried_2019_2020.csv",index=False)

### chrisp

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2019,1,1),datetime.datetime(2020,1,1)
aoi = ried
sm = chirps.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    data_aoi['geometry'] = aoi.getInfo()['features']
    data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][22:31],'%Y%m%d')
    df = pd.DataFrame(data_aoi)
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))
df_all.to_csv("chirps_ried_2019_2020.csv",index=False)


## GLDAS 2.1

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2010,1,1),datetime.datetime(2010,6,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2010_1.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2010,6,1),datetime.datetime(2011,1,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2010_2.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2011,1,1),datetime.datetime(2011,6,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2011_1.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2011,6,1),datetime.datetime(2012,1,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2011_2.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2012,1,1),datetime.datetime(2012,6,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2012_1.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2012,6,1),datetime.datetime(2013,1,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2012_2.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2013,1,1),datetime.datetime(2013,6,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2013_1.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2013,6,1),datetime.datetime(2014,1,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2013_2.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2014,1,1),datetime.datetime(2014,6,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2014_1.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2014,6,1),datetime.datetime(2015,1,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2014_2.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2015,1,1),datetime.datetime(2015,6,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2015_1.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2015,6,1),datetime.datetime(2016,1,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2015_2.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2016,1,1),datetime.datetime(2016,6,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2016_1.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2016,6,1),datetime.datetime(2017,1,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2016_2.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2017,1,1),datetime.datetime(2017,6,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2017_1.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2017,6,1),datetime.datetime(2018,1,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2017_2.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2018,1,1),datetime.datetime(2018,6,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2018_1.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2018,6,1),datetime.datetime(2019,1,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2018_2.csv",index=False)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2019,1,1),datetime.datetime(2019,6,1)
aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_ried_2019_1.csv",index=False)

In [ ]:
#Geometrys, Regions
points = ee.FeatureCollection([
            ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797)),
            ee.Feature(ee.Geometry.Point(8.241221721890724,49.6585087644599)),
            ])
point = ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797))

#Time
start,end = datetime.datetime(2015,1,1),datetime.datetime(2016,1,1)
#Source Dataset
chirps = chirps_precipitation.filterDate(start,end)
# Function Convert a FeatureCollection into a pandas DataFrame; Features is a list of dict with the output
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']  # GeoJSON Feature!
        #attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)

    df = gpd.GeoDataFrame(dictarr)
    # Convert GeoJSON features to shape
    #df['geometry'] = map(lambda x: shape(x), df.geometry)    
    return df

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, pts):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegions(collection=pts, reducer=ee.Reducer.mean(), scale=1000)
    # Convert to Pandas Dataframe
    df_image_red = fc2df(fc_image_red)
    # Add Date as Variable
    df_image_red['date'] = datetime.datetime.strptime(image.getInfo()['id'][22:31],'%Y%m%d')
    print(df_image_red)
    return df_image_red

#List of image propertys 
chirps_prop = surface_moisture.first().propertyNames().getInfo()
print('Check 1')
#Get propertie from every image to a List
chirps_id = [] #empty list
chirps_id = [item.get('id') for item in chirps.getInfo().get('features')]
#### Create Initial Pandas Dataframe
df_all = extract_point_values(chirps_id[0], points)
df_all = df_all.drop([0,1])
print("check 3")
#### Iterate over all impages
c=0
for i in chirps_id:
    c = c+ 1
    print("c",c)
    df_all = df_all.append(extract_point_values(i, points))
print("check 4")
#### Display Results
pp.pprint(df_all)
rad_point1 = df_all.loc[0]
rad_point2 = df_all.loc[1]


In [ ]:
import altair as alt
alt.renderers.enable('notebook')
pp.pprint(sm_point1['date'])
rad_point1['mean'] = rad_point1['mean'].multiply(4)
chart_sm = alt.Chart(sm_point1).mark_point().encode(x='date', y= 'smp')
chart_pp = alt.Chart(rad_point1).mark_point().encode(x='date', y='mean')
#alt.vconcat(chart_sm, chart_pp)
chart_sm + chart_pp

In [ ]:
pp.pprint(rad_point1)
pp.pprint(sm_point1)

In [ ]:
import altair as alt
data = 
chart = alt.Chart(data)

## Soil Moisture from Sentinel 1

In [ ]:
#Derived from Sentinel 1 A/B 2014-10 - Present
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')


In [ ]:
#Command Collection
.size().getInfo() # Get collection size
.bandNames().getInfo() # Get List with alls Bands from ee.Image not ImageCollection
.geometry().bounds().getInfo() # get Geometry of a Feature // Get bounding box of this geometry
.limit(5000) # Limit to the first 5000 Elements/Features

#Vectorizing
var vectors = image.reduceToVectors({
  geometry: FeatureCollection,
  crs: image.projection(),
  scale: 1000,
  geometryType: 'polygon',
  eightConnected: false,
  labelProperty: 'zone',
  reducer: ee.Reducer.mean()
});

#Masking
.clip(feature)
var image = ee.Image
var mask = image.gte(2).and(lt(5))
var maskedImage = image.updateMask(mask)

#Visualizations
thumbnail_url = image.getThumbUrl({
    'bands' : '',
    'min' : ,
    'max' : ,
    'region' : .geometry().bounds().getInfo() #must be a geojson 
})
IPython.display.HTML('Thumnail URL: <a href={0}>{0}</a>'.format(thumbnail_url)) #create url to view
IPython.display.Image(url=thumbnail_url) # view direct in notebook

#Interactive Visualizations
import ipyleaflet
map1 = ipyleaflet.Map(zoom=3, layout={'height' : '400px'})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1
dc.last_draw # gives information about the last drawn polygon (coordinates etc.)

#Function to create a tile layer urlfrom an gee image object
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

#style the image
tile_url = GetTileLayerUrl(image.visualize(min=0, max=3000, gamma=1.5, bands=['','','']))
map1.add_layer(ipyleaflet.TileLayer(url=tile_url))
#or create layer groups 

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import ee
from ipygee import *


def extract_time_series(start,end,coll,ried): #,sf

    
    # Obtain image collection for all images within query dates
    coll = coll.filterDate(start,end)

    # Get list of images which correspond with the above
    images = [item.get('id') for item in coll.getInfo().get('features')]
    store = []
    date_store = []
    print(images)
    # Loop over all images and extract pixel value
    for image in images:
        
        im = ee.Image(image)
        #projection = im.projection().getInfo()['crs']
        # Obtain date from timestamp in metadata
        date = dt.fromtimestamp(im.get("system:time_start").getInfo() / 1000.)
        date_store.append(np.datetime64(date))

        # Extract pixel value
        data = im.reduceRegion(ee.Reducer.mean(),ried, 1000) #,1, crs=projection).get(band_name) 
        store.append(data.getInfo())
        print(store)
    # Scale the returned data based on scale factor
    #store = [x * sf if isinstance(x, int) else np.nan for x in store]
    
    # Convert output into pandas data frame
    df = pd.DataFrame(index=date_store, data=store, columns=['precipitation'])
    df['store']
    return df


band_name = 'b1'
coll = chirps_precipitation
start,end = datetime.datetime(2015,1,1),datetime.datetime(2015,1,5)
# Set up point geometry
points = ee.FeatureCollection([
        ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797)),
        ee.Feature(ee.Geometry.Point(8.241221721890724,49.6585087644599)),
        ])
point = ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797))


rad_data = extract_time_series(start,end,coll,ried)
print(rad_data)